# CARF Router Training (Colab)

This notebook generates Cynefin router training data with DeepSeek,
trains a DistilBERT classifier, and evaluates the model.

Notes:
- You must provide your own DEEPSEEK_API_KEY.
- Use GPU in Colab for faster training (Runtime -> Change runtime type).


## 1) Optional GPU check


In [14]:
import shutil
import subprocess

if shutil.which("nvidia-smi"):
    subprocess.run(["nvidia-smi"])
else:
    print("nvidia-smi not found. GPU not enabled.")

try:
    import torch
    print("torch cuda available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("cuda device:", torch.cuda.get_device_name(0))
except Exception as exc:
    print("torch not installed yet:", exc)


nvidia-smi not found. GPU not enabled.
torch cuda available: False


## 2) Load the repo
Set REPO to your GitHub repo. If the repo is private, use a GitHub token.


In [15]:
# Option A: clone from Git (supports private repos via token)
import os
import getpass
import subprocess

REPO = 'eljasuhonen/projectcarfcynepic'
REPO_DIR = 'projectcarfcynepic'

def _clone_with_token(token_value):
    repo_url = f"https://github.com/{REPO}.git"
    clone_args = ["git", "clone", repo_url, REPO_DIR]
    if token_value:
        clone_args = [
            "git",
            "-c",
            f"http.extraHeader=AUTHORIZATION: bearer {token_value}",
            "clone",
            repo_url,
            REPO_DIR,
        ]
    subprocess.run(clone_args, check=True)

token = os.getenv("GITHUB_TOKEN")
if token is None:
    token = getpass.getpass('GITHUB_TOKEN (leave empty if repo is public): ').strip()

if not os.path.isdir(REPO_DIR):
    try:
        _clone_with_token(token)
    except subprocess.CalledProcessError as exc:
        if not token:
            print("Clone failed. Repo might be private. Retrying with token...")
            token = getpass.getpass("GITHUB_TOKEN: ").strip()
            _clone_with_token(token)
        else:
            raise exc

os.chdir(REPO_DIR)
print(f"Working directory: {os.getcwd()}")


Clone failed. Repo might be private. Retrying with token...


CalledProcessError: Command '['git', 'clone', 'https://github.com/eljasuhonen/projectcarfcynepic.git', 'projectcarfcynepic']' returned non-zero exit status 128.

In [ ]:
# Option B: mount Drive and point to your repo
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/projectcarf


## 3) Install dependencies


In [4]:
!pip install -e ".[router]"


Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


## 4) Set API key


In [ ]:
import os
import getpass

os.environ['LLM_PROVIDER'] = 'deepseek'
os.environ['DEEPSEEK_API_KEY'] = getpass.getpass('DEEPSEEK_API_KEY: ')


## 5) Generate training data
Adjust --examples-per-domain as needed.


In [ ]:
!python scripts/generate_router_training_data.py \n
    --examples-per-domain 200 \n
    --batch-size 50


## 6) Train DistilBERT


In [ ]:
!python scripts/train_router_model.py \n
    --data-file data/router_training/cynefin_router_training.jsonl \n
    --output-dir models/router_distilbert


## 7) Evaluate the model


In [ ]:
!python scripts/evaluate_router_model.py \n
    --data-file data/router_training/cynefin_router_training.jsonl \n
    --model-path models/router_distilbert


## 8) Use the model locally
Set env vars before starting the backend:

ROUTER_MODE=distilbert
ROUTER_MODEL_PATH=models/router_distilbert
